In [1]:
%cd ~/user_data/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base
%pwd

/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base


'/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base'

In [2]:
from torchvision.datasets import CocoDetection
import pandas as pd
import torch
import json

In [3]:
val_image_dir = r'/mnt/active_storage/Joe/coco_set/val2017/images'
val_annotation_path = r'/mnt/active_storage/Joe/coco_set/val2017/annotations/ovd_ins_val2017_b.json'

coco_val_dataset = CocoDetection(
    root=val_image_dir, 
    annFile=val_annotation_path,
)

loading annotations into memory...
Done (t=0.66s)
creating index...
index created!


In [4]:
ovd_base_image_ids = []
for item in coco_val_dataset:
    ovd_base_image_ids.append(item[1][0]['image_id'])

In [5]:
category_names = ['person', 'bicycle', 'car', 'motorcycle', 'train', 'truck', \
    'boat', 'bench', 'bird', 'horse', 'sheep', 'bear', 'zebra', 'giraffe', \
    'backpack', 'handbag', 'suitcase', 'frisbee', 'skis', 'kite', 'surfboard', \
    'bottle', 'fork', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', \
    'broccoli', 'carrot', 'pizza', 'donut', 'chair', 'bed', 'toilet', 'tv', \
    'laptop', 'mouse', 'remote', 'microwave', 'oven', 'toaster', \
    'refrigerator', 'book', 'clock', 'vase', 'toothbrush']

In [6]:
predictions_df_total = pd.read_parquet(r'/mnt/archive/owlvit_results/owlv2_coco_base_predictions.parquet')

In [7]:
objectness_threshold = 0.2
score_threshold = 0.1

predictions_likely_objects = predictions_df_total[predictions_df_total['objectness_score'] >= objectness_threshold].sort_values('objectness_score', ascending=False).groupby('image_id').head(100).sort_values('image_id')
predictions_likely_objects = predictions_likely_objects.loc[predictions_likely_objects['image_id'].isin(ovd_base_image_ids)]

known_objects = predictions_likely_objects[predictions_likely_objects['score'] >= score_threshold]

display(known_objects)

,image_id,label,score,objectness_score,xmin,ymin,xmax,ymax
1233,139,vase,0.351422,0.227661,350.876923,209.574448,362.114807,230.386261
1336,139,vase,0.334456,0.284153,166.681076,232.826904,186.017776,266.831390
1150,139,tv,0.704314,0.542719,5.215874,166.488068,154.236847,267.109009
1163,139,vase,0.248859,0.221027,240.760803,197.673264,253.225784,213.093719
1253,139,tv,0.409905,0.306233,559.908569,209.109192,640.466003,289.297943
...,...,...,...,...,...,...,...,...
178952,581781,orange,0.176230,0.327590,281.273376,434.908295,371.029541,478.781433
178161,581781,banana,0.284449,0.241578,201.138290,274.905090,270.746216,331.941162
177593,581781,banana,0.301017,0.333404,445.379303,101.757217,638.398376,269.523865
178102,581781,banana,0.323884,0.339627,141.406525,184.186020,441.437103,460.155182


In [8]:
known_image_ids = known_objects['image_id'].unique().tolist()

In [9]:
adam_classifications = torch.load(r'./output_with_scores.pt')

In [10]:
print(adam_classifications[0])

{'image_id': 139, 'boxes': tensor([[350.8769, 209.5744, 362.1148, 230.3863],
        [166.6811, 232.8269, 186.0178, 266.8314],
        [  5.2159, 166.4881, 154.2368, 267.1090],
        [240.7608, 197.6733, 253.2258, 213.0937],
        [559.9086, 209.1092, 640.4660, 289.2979],
        [549.7681, 306.8633, 586.3107, 400.1674],
        [555.1292, 290.8761, 577.7758, 331.9653],
        [466.0369, 354.4450, 637.8002, 425.3321],
        [337.8965, 176.3780, 381.8003, 222.0279],
        [  2.1859,   2.0000,   2.3986,   1.0000],
        [505.3746,  84.2601, 534.2266, 135.9551],
        [120.8437, 356.6360, 466.8375, 426.6800],
        [573.5719, 249.9596, 606.8554, 309.2519]]), 'labels': tensor([33, 33, 43, 43, 43, 43, 30, 45, 43, 32, 21, 46, 47], dtype=torch.int32), 'scores': tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.3367, 0.9678, 0.4067,
        0.6875, 0.4932, 0.9614, 0.4680])}


In [11]:
adam_image_ids = [adam_classification['image_id'] for adam_classification in adam_classifications]

In [12]:
print(type(adam_classifications[0]['image_id']))

<class 'int'>


In [13]:
known_only_ids = list(set(known_image_ids) - set(adam_image_ids))
for idx, id in enumerate(known_only_ids):
    known_only_ids[idx] = int(known_only_ids[idx])

In [14]:
print(len(known_only_ids))

640


In [15]:
print(len(adam_classifications))

3892


In [16]:
combined_classifications = adam_classifications.copy()

for known_only_id in known_only_ids:
    combined_classifications.append({
        'image_id': int(known_only_id),
        'boxes': [],
        'labels': [],
        'scores': []
    })

In [17]:
for row_idx, row in known_objects.iterrows():
    # Get image_id of this annotation
    row_image_id = int(row['image_id'])
    # iterate through all the classifications in the list so we can index correctly
    for img_idx, image_data in enumerate(combined_classifications):
        if  image_data['image_id'] == row_image_id:
            for score_idx, score in enumerate(combined_classifications[img_idx]['scores']):
                if (score == 1.0 
                and float(f"{row['xmin']:.4f}") == float(f"{combined_classifications[img_idx]['boxes'][score_idx][0]:.4f}") 
                and float(f"{row['ymin']:.4f}") == float(f"{combined_classifications[img_idx]['boxes'][score_idx][1]:.4f}")
                and float(f"{row['xmax']:.4f}") == float(f"{combined_classifications[img_idx]['boxes'][score_idx][2]:.4f}")
                and float(f"{row['ymax']:.4f}") == float(f"{combined_classifications[img_idx]['boxes'][score_idx][3]:.4f}")):
                    combined_classifications[img_idx]['scores'][score_idx] = row['score']
            if row_image_id in known_only_ids:
                combined_classifications[img_idx]['boxes'].append([row['xmin'], row['ymin'], row['xmax'], row['ymax']])
                combined_classifications[img_idx]['labels'].append(category_names.index(row['label']))
                combined_classifications[img_idx]['scores'].append(row['score'])

In [18]:
for idx, classification in enumerate(combined_classifications):
    if type(classification['boxes']) == type([]):
        combined_classifications[idx]['boxes'] = torch.tensor(combined_classifications[idx]['boxes'], dtype=torch.int)
        combined_classifications[idx]['labels'] = torch.tensor(combined_classifications[idx]['labels'], dtype=torch.int)
        combined_classifications[idx]['scores'] = torch.tensor(combined_classifications[idx]['scores'], dtype=torch.float)

In [24]:
predicted_image_ids = [item['image_id'] for item in combined_classifications]
missing_image_ids = list(set(ovd_base_image_ids) - set(predicted_image_ids))

for missing_image_id in missing_image_ids:
    combined_classifications.append({
        'image_id': missing_image_id,
        'boxes': torch.tensor([], dtype=torch.int),
        'labels': torch.tensor([], dtype=torch.int),
        'scores': torch.tensor([], dtype=torch.int)
    })

In [25]:
assert len(combined_classifications) == len(coco_val_dataset), "The classifications list is missing images from the ground truth"

In [29]:
torch_write_path = r'/mnt/archive/owlvit_results/coco_base_obj2cls1_combined_prepped_for_torchmetrics.pt'
torch.save(combined_classifications, torch_write_path)

In [30]:
loaded_torch = torch.load(torch_write_path)

In [31]:
assert len(combined_classifications) == len(loaded_torch), "Saved and loaded lists are different lengths"

for idx in range(len(combined_classifications)):
    assert combined_classifications[idx]['image_id'] == loaded_torch[idx]['image_id'], f"Mismatched image ids at index {idx}"
    assert torch.equal(combined_classifications[idx]['boxes'], loaded_torch[idx]['boxes']), f"Mismatched boxes at index {idx}"
    assert torch.equal(combined_classifications[idx]['labels'], loaded_torch[idx]['labels']), f"Mismatched labels at index {idx}"
    assert torch.equal(combined_classifications[idx]['scores'], loaded_torch[idx]['scores']), f"Mismatched scores at index {idx}"